<a href="https://colab.research.google.com/github/machine-perception-robotics-group/JDLALectureNotebooks/blob/master/notebooks/xx_multi_label_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ニューラルネットワークによるマルチラベル分類

 ---

 ## 目的

 ニューラルネットワークを使用して，マルチラベル画像分類を行う．

 # モジュールのインポート

 プログラムの実行に必要なモジュールをインポートします．

In [ ]:
import os
from PIL import Image

import torch
import torch.nn as nn

from torchvision import transforms

import csv

## データセットのダウンロード

今回のデータセットには1枚の画像に複数個のラベルが付与されたデータセットです．

[Multi-label Image Classification Dataset (Kaggle)](https://www.kaggle.com/datasets/meherunnesashraboni/multi-label-image-classification-dataset)

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=1mOsi95Nj-s6gAlp25PPoRY6hpvwFPgTE', 'multilabel_modified.zip', quiet=False)
! unzip -q multilabel_modified.zip

### ダウンロードしたデータセットの確認

ダウンロードしたデータセットのフォルダ内には，画像が格納されたimagesフォルダとlabelファイルのCSVファイルがあります．

このうち，labels.csvを開いて中身を確認してみます．

ファイルには，画像ファイル名，対応するクラス名，01で表現された各クラスに対するラベルが格納されいています．

存在するクラスはtruck, boat, busなどの計16種類です．

In [ ]:
with open('multilabel_modified/labels.csv', 'r') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        print(row)
        if i !=0:
            print(list(map(int, row[2:])))
        if i > 5:
            break

## データセットクラスの作成

上で確認したデータを元にPyTorch用のデータセットクラスを作成します．

csvファイルには8000サンプル以上の情報が格納されていますが，ここで使用するサンプルは7843枚となっているため，`__len__`の数を明示的に指定しています．

In [ ]:
class MultilabelDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, root_dir):
        self.root_dir = root_dir
        self.data = []
        self.label = []
        self.trans = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        with open(csv_file, 'r') as f:
            reader = csv.reader(f)
            next(reader) # headerをスキップ
            for row in reader:
                self.data.append(row[0])
                self.label.append(list(map(int, row[2:])))

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.data[index])
        img = Image.open(img_path)
        img = self.trans(img)
        label = torch.tensor(self.label[index])
        return img, label

    def __len__(self):
        return 7843

## ネットワークの作成

畳み込みニューラルネットワークを作成します．

In [ ]:
class MultilabelCNN(nn.Module):
    def __init__(self, n_classes=16):
        super(MultilabelCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, n_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## 学習

作成したデータセットを使用してネットワークモデルを学習します．

In [ ]:
# デバイスの設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# データセットの作成
train_dataset = MultilabelDataset('multilabel_modified/labels.csv', 'multilabel_modified/images')
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# モデルの作成
model = MultilabelCNN().to(device)

# 損失関数とオプティマイザ
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 学習
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels.float())

        # 逆伝播と最適化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')